In [1]:
import os 
import json
import pprint
%matplotlib inline
from serializer import*
import tweepy
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
from pyvis.network import Network
import os

#Twitter API credentials
api_key = "ENFvcEFBU6prbQWZVMKYHhHCW"
api_secret = "OoLxYiucfTgnheUH1Bp7YJTsyIto9VcxLERBeXCg84NwjjgCAr"
access_token = "2572511777-adhDu1rnH5RYVPdfk51jPzLbOzpppivKmVYTtFH"
access_secret = "Oq1Q8CTYoDKxfyal7aY2x5V88QBqM2SAFac5e8LV6eeWY"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAJkTJQEAAAAAaoRAjF4yb5OBca0iPYQZvs5vH5s%3D5AG5ZaEGM6pFtGL7cQohdtzimqd1aLcqHGbJPOSCXO4Fxnltwz"

In [2]:
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, timeout = 120)
if(api.verify_credentials):
    print ('Authentication completed successfully!')
    
s = Serializer("data")

Authentication completed successfully!


In [ ]:
users = ["mizzaro"]

for user in users:
    print(f"Processing @{user}")

    user_followers = []

    for item in tweepy.Cursor(
            api.friends,
            screen_name=user,
            skip_status=True,
            include_user_entities=False
    ).items():
        user_followers.append(item._json)

    print(f"Found {len(user_followers)} following for @{user}")
    s.serialize_json(f"{user}/{user}_following.json", user_followers)

Processing @mizzaro


Rate limit reached. Sleeping for: 892


In [14]:
###############################OLD###################################
import os

directory = "data"
G = nx.DiGraph(membri=["Lorenzo Bellina" "Francesco Bombassei De Bona", "Andrea Cantarutti", "Gabriele Dominici"])
for dirname in os.listdir(directory):
    if not dirname.startswith(".") and not dirname == "all_users.json":
        print(f"Analizzo {dirname}")
        s = Serializer(f"data/{dirname}")
        profile = s.read_json(f"{dirname}_profile.json")
        G.add_node(profile["id"], **profile)
        print(f'Aggiungo il nodo {profile["name"]}')

        followers = s.read_json(f"{dirname}_followers.json")
        print(f'Aggiungo i {len(followers)} followers di {profile["name"]}')
        for item in followers:
            G.add_node(item["id"], **item)
            G.add_edge(item["id"], profile["id"])

        following = s.read_json(f"{dirname}_following.json")
        print(f'Aggiungo i {len(following)} following di {profile["name"]}')
        for item in following:
            G.add_node(item["id"], **item)
            G.add_edge(profile["id"], item["id"])

        print(f'Inserisco i random di {profile["name"]}')
        temp_dir = directory+"/"+dirname
        for file in os.listdir(temp_dir):
            if file.startswith("random"):
                id_user = int(file.split("_")[1])
                if file.endswith("following.json"):
                    following = s.read_json(f"{file}")
                    print(f"Inserisco i {len(following)} following del random di id {id_user}")
                    for item in following:
                        G.add_node(item["id"], **item)
                        G.add_edge(id_user, item["id"])
                elif file.endswith("follower.json"):
                    followers = s.read_json(f"{file}")
                    print(f"Inserisco i {len(followers)} followers del random di id {id_user}")
                    for item in followers:
                        G.add_node(item["id"], **item)
                        G.add_edge(item["id"], id_user)
        node_it = G.in_degree()
        for (node, in_degree) in node_it:
            G.nodes[node]["follower_finded"] = in_degree 
            
nx.write_gpickle(G, "graph/graph_networkx.pkl")

Analizzo all_friendships.json
No data found at path: data/all_friendships.json/all_friendships.json_profile.json


KeyError: 'id'

In [4]:
def get_friendship(sourceid, targetid, api):
    kind = ""

    friendship = api.show_friendship(source_id=sourceid, target_id=targetid)

    if not friendship[0].following and not friendship[0].followed_by:
        kind = "none"
    elif not friendship[0].following and friendship[0].followed_by:
        kind = "r_l"
    elif friendship[0].following and not friendship[0].followed_by:
        kind = "l_r"
    else:
        kind = "bi"

    return {
        "source_id": sourceid,
        "target_id": targetid,
        "friendship": kind
    }

accounts = ["Miccighel_"]
serializer = Serializer("data")
users = serializer.read_json("all_users.json")
edges = []
for account in accounts:
    serializer = Serializer(f'data/{account}')
    account_json = serializer.read_json(f"{account}_profile.json")
    account_id = account_json["id"]
    for user in users:
        if user["id"] is not account_id:
            friendship = get_friendship(account_id, user["id"], api)
            edges.append(friendship)
            if friendship["friendship"] != "none":
                print(friendship)
            print(f"Added friendship between {account} and {user['screen_name']}")
    serializer.serialize_json(f'{account}_friendships.json', edges)

Data read from path: data/all_users.json
Data read from path: data/Miccighel_/Miccighel__profile.json
Added friendship between Miccighel_ and Diana_13721
Added friendship between Miccighel_ and LostQue25565910
Added friendship between Miccighel_ and micros2021
Added friendship between Miccighel_ and PopMan90643879
Added friendship between Miccighel_ and colleen_chen_
Added friendship between Miccighel_ and manafiali1
{'source_id': 15750573, 'target_id': 14616323, 'friendship': 'r_l'}
Added friendship between Miccighel_ and Duesynapse
Added friendship between Miccighel_ and MsaiedurR
Added friendship between Miccighel_ and covid19mlia
Added friendship between Miccighel_ and IshratRahmanSa1
Added friendship between Miccighel_ and Dr_Oli_Jones
Added friendship between Miccighel_ and Hamdy_Mubarak
Added friendship between Miccighel_ and aleksbobic92
Added friendship between Miccighel_ and jakusg
Added friendship between Miccighel_ and jeanburgess
Added friendship between Miccighel_ and Dis

Added friendship between Miccighel_ and datafuturology
Added friendship between Miccighel_ and trevi
Added friendship between Miccighel_ and maik_froebe
Added friendship between Miccighel_ and __alucic
Added friendship between Miccighel_ and tuantx7110
Added friendship between Miccighel_ and salias
Added friendship between Miccighel_ and shahrzad_naseri
Added friendship between Miccighel_ and deb_mazu
Added friendship between Miccighel_ and SIGIReCom
Added friendship between Miccighel_ and FabianoDeiss
Added friendship between Miccighel_ and diazf_acm
Added friendship between Miccighel_ and FattaneZ
Added friendship between Miccighel_ and d_anubrata
Added friendship between Miccighel_ and mdekstrand
Added friendship between Miccighel_ and iFromm
Added friendship between Miccighel_ and vitordouzi
Added friendship between Miccighel_ and harlleyaugusto
Added friendship between Miccighel_ and tab_larasa
Added friendship between Miccighel_ and pchandarr
Added friendship between Miccighel_ a

Rate limit reached. Sleeping for: 847


Added friendship between Miccighel_ and earissola
Added friendship between Miccighel_ and JToscane
Added friendship between Miccighel_ and OkwenAkah
Added friendship between Miccighel_ and asia_biega
Added friendship between Miccighel_ and miriam_fs
Added friendship between Miccighel_ and UNEDNLP
Added friendship between Miccighel_ and MIT_CSAIL
Added friendship between Miccighel_ and kallumadi
Added friendship between Miccighel_ and panglos00047743
Added friendship between Miccighel_ and HarryButcher8
Added friendship between Miccighel_ and Hu_Hsu
Added friendship between Miccighel_ and TAMANNA80943294
Added friendship between Miccighel_ and gsingers
Added friendship between Miccighel_ and dorrreen
Added friendship between Miccighel_ and phillips_bills
Added friendship between Miccighel_ and CheiaJulio
Added friendship between Miccighel_ and KieselJohannes
Added friendship between Miccighel_ and rzhangpku
Added friendship between Miccighel_ and JamesMariolle
Added friendship between M

Added friendship between Miccighel_ and _nhos_
Added friendship between Miccighel_ and hvdthong
Added friendship between Miccighel_ and foclasa2017
Added friendship between Miccighel_ and maliannejadi
Added friendship between Miccighel_ and YashMosh
Added friendship between Miccighel_ and alvarory_
Added friendship between Miccighel_ and socinformatics
Added friendship between Miccighel_ and shashank27392
Added friendship between Miccighel_ and ICTIR17
Added friendship between Miccighel_ and essir2017
Added friendship between Miccighel_ and GunturuManohar
Added friendship between Miccighel_ and irnewsjapan
Added friendship between Miccighel_ and CAIRWorkshop
Added friendship between Miccighel_ and IAI_group
Added friendship between Miccighel_ and bernn
Added friendship between Miccighel_ and HaythamFayek
Added friendship between Miccighel_ and femoraesg
Added friendship between Miccighel_ and kferris01
Added friendship between Miccighel_ and A_D_Delgado
Added friendship between Miccigh

...

Rate limit reached. Sleeping for: 851


Added friendship between Miccighel_ and grazia_borri
Added friendship between Miccighel_ and CitImmCanada
Added friendship between Miccighel_ and Caradelevingne
Added friendship between Miccighel_ and CarasThighGap
Added friendship between Miccighel_ and luke_brooks
Added friendship between Miccighel_ and janoskians
Added friendship between Miccighel_ and EB_amaramavera
Added friendship between Miccighel_ and AceYOLO8
Added friendship between Miccighel_ and JJwright89
Added friendship between Miccighel_ and Miikluka
Added friendship between Miccighel_ and DrDavisVang
Added friendship between Miccighel_ and Iluvhorsepower
Added friendship between Miccighel_ and Joinponrs
Added friendship between Miccighel_ and jackjordan478
Added friendship between Miccighel_ and GrowingUpADHD
Added friendship between Miccighel_ and MiamisModels
Added friendship between Miccighel_ and JimmyVannawong2
Added friendship between Miccighel_ and jdeschai1
Added friendship between Miccighel_ and Khanya61447802

Added friendship between Miccighel_ and peterpaws
Added friendship between Miccighel_ and wesselkraaij
Added friendship between Miccighel_ and tonygrr
Added friendship between Miccighel_ and donadeb
Added friendship between Miccighel_ and UNED
Added friendship between Miccighel_ and bing
Added friendship between Miccighel_ and marcotravaglio
Added friendship between Miccighel_ and ericschmidt
Added friendship between Miccighel_ and SabinaGuzzanti
Added friendship between Miccighel_ and PrestaMarco
Added friendship between Miccighel_ and williamhersh
Added friendship between Miccighel_ and yudapearl
Added friendship between Miccighel_ and GuillaumeSalha
Added friendship between Miccighel_ and m_zecchin
Added friendship between Miccighel_ and alevalsecchi
Added friendship between Miccighel_ and effecube
Added friendship between Miccighel_ and Riccardi_FVG
Added friendship between Miccighel_ and fissore_f
Added friendship between Miccighel_ and SergioBini68


Rate limit reached. Sleeping for: 850


Added friendship between Miccighel_ and AFlaccilla
Added friendship between Miccighel_ and CiriSince1978
Added friendship between Miccighel_ and giuzan
Added friendship between Miccighel_ and vuotodaria_
Added friendship between Miccighel_ and indiasardana
Added friendship between Miccighel_ and wilfredbulus
Added friendship between Miccighel_ and ELYgonzalez0822
Added friendship between Miccighel_ and nick_hoffman8
Added friendship between Miccighel_ and Mz_DesiPie
Added friendship between Miccighel_ and DonLs71
Added friendship between Miccighel_ and STetelo
Added friendship between Miccighel_ and vossie23
Added friendship between Miccighel_ and f_altruist
Added friendship between Miccighel_ and BSakhakhini
Added friendship between Miccighel_ and zubair_1811
Added friendship between Miccighel_ and Fnalhayan
Added friendship between Miccighel_ and Julian_Risch
Added friendship between Miccighel_ and HPCLab
Added friendship between Miccighel_ and 9meo
Added friendship between Miccighel

In [11]:
####################CON ALL_USER E ALL_FRIENDSHIP######################
directory = "check_data"
G = nx.Graph(membri=["Lorenzo Bellina" "Francesco Bombassei De Bona", "Andrea Cantarutti", "Gabriele Dominici"])
s = Serializer(f"{directory}")
all_nodes = s.read_json(f"unique_all_users.json")
double = []
print(len(list(all_nodes)))
for profile in all_nodes:
    if profile["id"] in G.nodes:
        double.append(profile["id"])
    G.add_node(profile["id"], follower_ing = 0, following_ing = 0, **profile)
    #print(f'Aggiungo il nodo {profile["name"]}')
s.serialize_json("double_users.json", double)
print(len(list(G.nodes)))
all_edges = s.read_json(f"all_friendships.json")
miss = []
for friendship in all_edges:
    if(friendship["friendship"] != "none"):
        if friendship["source_id"] not in G.nodes:
            miss.append(friendship["source_id"])
        if friendship["target_id"] not in G.nodes:
            miss.append(friendship["target_id"])
        G.add_edge(friendship["source_id"], friendship["target_id"], value = friendship["friendship"])
        print(f'Aggiungo l arco tra {friendship["source_id"]} e {friendship["target_id"]}')
s.serialize_json("missed_users.json", miss)
print(len(list(G.nodes)))

for edge in G.edges:
    if G[edge[0]][edge[1]]["value"] == "bi":
        G.nodes[edge[0]]["follower_ing"] += 1
        G.nodes[edge[0]]["following_ing"] += 1
        G.nodes[edge[1]]["follower_ing"] += 1
        G.nodes[edge[1]]["following_ing"] += 1
    elif G[edge[0]][edge[1]]["value"] == "r_l":
        G.nodes[edge[0]]["follower_ing"] += 1
        G.nodes[edge[1]]["following_ing"] += 1
    elif G[edge[0]][edge[1]]["value"] == "l_r":
        G.nodes[edge[1]]["follower_ing"] += 1
        G.nodes[edge[0]]["following_ing"] += 1

nx.draw(G)
print(len(G.nodes))
print(len(G.edges))

nx.write_gpickle(G, "graph/graph_networkx.pkl")

Data read from path: check_data/unique_all_users.json
3380
Data serialized to path: check_data/double_users.json
3217
Data read from path: check_data/all_friendships.json
Aggiungo l arco tra 19659370 e 132646210
Aggiungo l arco tra 19659370 e 3036907250
Aggiungo l arco tra 19659370 e 1556357336
Aggiungo l arco tra 19659370 e 262236415
Aggiungo l arco tra 19659370 e 813286
Aggiungo l arco tra 19659370 e 1111315973125169152
Aggiungo l arco tra 19659370 e 44196397
Aggiungo l arco tra 19659370 e 15750573
Aggiungo l arco tra 19659370 e 73001375
Aggiungo l arco tra 19659370 e 333597222
Aggiungo l arco tra 19659370 e 569203915
Aggiungo l arco tra 19659370 e 2336440088
Aggiungo l arco tra 19659370 e 1276832114856656896
Aggiungo l arco tra 19659370 e 1214864022572425217
Aggiungo l arco tra 19659370 e 877108837475655680
Aggiungo l arco tra 19659370 e 51934480
Aggiungo l arco tra 19659370 e 2807461219
Aggiungo l arco tra 19659370 e 268025507
Aggiungo l arco tra 19659370 e 249255635
Aggiungo l arc

Aggiungo l arco tra 132646210 e 10313732
Aggiungo l arco tra 132646210 e 14614664
Aggiungo l arco tra 132646210 e 190006526
Aggiungo l arco tra 132646210 e 10868922
Aggiungo l arco tra 132646210 e 190821474
Aggiungo l arco tra 132646210 e 1441677282
Aggiungo l arco tra 132646210 e 17354555
Aggiungo l arco tra 132646210 e 17657107
Aggiungo l arco tra 132646210 e 17730501
Aggiungo l arco tra 132646210 e 281079339
Aggiungo l arco tra 132646210 e 279084731
Aggiungo l arco tra 132646210 e 103810205
Aggiungo l arco tra 132646210 e 242868086
Aggiungo l arco tra 132646210 e 234017215
Aggiungo l arco tra 132646210 e 73719913
Aggiungo l arco tra 132646210 e 26711278
Aggiungo l arco tra 132646210 e 422016668
Aggiungo l arco tra 132646210 e 1077633912
Aggiungo l arco tra 132646210 e 117553058
Aggiungo l arco tra 132646210 e 953752657
Aggiungo l arco tra 132646210 e 438301250
Aggiungo l arco tra 132646210 e 27104786
Aggiungo l arco tra 132646210 e 861246336
Aggiungo l arco tra 132646210 e 126274094

Aggiungo l arco tra 3036907250 e 78969930
Aggiungo l arco tra 3036907250 e 3761306416
Aggiungo l arco tra 3036907250 e 544737616
Aggiungo l arco tra 3036907250 e 2601589496
Aggiungo l arco tra 3036907250 e 4742002531
Aggiungo l arco tra 3036907250 e 15455450
Aggiungo l arco tra 3036907250 e 338989477
Aggiungo l arco tra 3036907250 e 485865743
Aggiungo l arco tra 3036907250 e 119512412
Aggiungo l arco tra 3036907250 e 59217922
Aggiungo l arco tra 3036907250 e 15378449
Aggiungo l arco tra 3036907250 e 11892372
Aggiungo l arco tra 3036907250 e 18932422
Aggiungo l arco tra 3036907250 e 3074437553
Aggiungo l arco tra 3036907250 e 2769983813
Aggiungo l arco tra 3036907250 e 305474242
Aggiungo l arco tra 3036907250 e 350164545
Aggiungo l arco tra 3036907250 e 18417153
Aggiungo l arco tra 3036907250 e 117483950
Aggiungo l arco tra 3036907250 e 2910644884
Aggiungo l arco tra 3036907250 e 125483940
Aggiungo l arco tra 3036907250 e 16868154
Aggiungo l arco tra 3036907250 e 1339637532
Aggiungo l a

Aggiungo l arco tra 18932422 e 17743823
Aggiungo l arco tra 18932422 e 91168371
Aggiungo l arco tra 18932422 e 2475224005
Aggiungo l arco tra 18932422 e 54875928
Aggiungo l arco tra 18932422 e 1339637532
Aggiungo l arco tra 18932422 e 3332231
Aggiungo l arco tra 18932422 e 25134469
Aggiungo l arco tra 18932422 e 19677430
Aggiungo l arco tra 18932422 e 130671142
Aggiungo l arco tra 18932422 e 356775978
Aggiungo l arco tra 18932422 e 332134740
Aggiungo l arco tra 18932422 e 271243941
Aggiungo l arco tra 18932422 e 375007888
Aggiungo l arco tra 18932422 e 1086604922
Aggiungo l arco tra 18932422 e 382158989
Aggiungo l arco tra 18932422 e 22026924
Aggiungo l arco tra 18932422 e 139811295
Aggiungo l arco tra 18932422 e 3107101390
Aggiungo l arco tra 18932422 e 5734242
Aggiungo l arco tra 18932422 e 8220532
Aggiungo l arco tra 18932422 e 799830497065050112
Aggiungo l arco tra 18932422 e 264501255
Aggiungo l arco tra 18932422 e 106856453
Aggiungo l arco tra 18932422 e 89774258
Aggiungo l arco 

KeyError: 'follower_ing'

In [39]:
connected = nx.is_connected(G)
print(connected)
bipartite = nx.is_bipartite(G)
print(bipartite)
"""
center = nx.center(G)
print(center)
diameter = nx.diameter(G)
print(diameter)
radius = nx.radius(G)
print(radius)
"""
#betweenness = nx.betweenness_centrality(G) #normalized=True default
#print(betweenness)
#closeness = nx.closeness_centrality(G) #wf=True default Wasserman and Faust improved - incremental closeness??
#print(closeness)
degree = nx.degree_centrality(G)
print(degree)
count = 0

for item in degree.items():
    if item[1] >= 0.0031:
        print(item[0])
        count += 1
        
print(count)
#in_degree = nx.in_degree_centrality(G)
#print(in_degree)
#out_degree = nx.out_degree_centrality(G)
#print(out_degree)
page_rank = nx.pagerank(G) #parametri extra
print(page_rank)
hits = nx.hits(G) #parametri extra
print(hits)

min_edge_cover = nx.min_edge_cover(G) #extra
print(min_edge_cover)

omega = nx.omega(G) #extra
print(omega)
sigma = nx.sigma(G) #extra
print(sigma)

False
False
{108207407: 0.00031959092361776926, 1319272146963673089: 0.00031959092361776926, 1313137113454579717: 0.0006391818472355385, 1301869598481014787: 0.00031959092361776926, 1046614848: 0.00031959092361776926, 2893424886: 0.0006391818472355385, 14616323: 0.0015979546180888463, 2842688136: 0.00031959092361776926, 1256240551118200835: 0.0006391818472355385, 1276832114856656896: 0.0006391818472355385, 301709965: 0.00031959092361776926, 632815919: 0.00031959092361776926, 748176824: 0.00031959092361776926, 1535439446: 0.00031959092361776926, 18202677: 0.0006391818472355385, 1233080567727521792: 0.00031959092361776926, 257111136: 0.00031959092361776926, 1232460508160094209: 0.00031959092361776926, 2981463028: 0.00031959092361776926, 2960317030: 0.00031959092361776926, 181348220: 0.00031959092361776926, 1369078711: 0.0006391818472355385, 22585506: 0.00031959092361776926, 48677032: 0.001278363694471077, 433873132: 0.00031959092361776926, 167735563: 0.00031959092361776926, 1081076050061

{108207407: 0.00014929238320231903, 1319272146963673089: 0.00014929238320231903, 1313137113454579717: 0.0002424061887877945, 1301869598481014787: 0.00014929238320231903, 1046614848: 0.00014929238320231903, 2893424886: 0.000246478571429373, 14616323: 0.0006044273109349298, 2842688136: 0.00014929238320231903, 1256240551118200835: 0.0002424061887877945, 1276832114856656896: 0.00025036202187502757, 301709965: 0.00014929238320231903, 632815919: 0.00014929238320231903, 748176824: 0.00014929238320231903, 1535439446: 0.00014929238320231903, 18202677: 0.00025036202187502757, 1233080567727521792: 0.00014929238320231903, 257111136: 0.00014929238320231903, 1232460508160094209: 0.00014929238320231903, 2981463028: 0.00014929238320231903, 2960317030: 0.00014929238320231903, 181348220: 0.00014929238320231903, 1369078711: 0.00024305562295050595, 22585506: 0.00014929238320231903, 48677032: 0.0004293815863287809, 433873132: 0.00014929238320231903, 167735563: 0.00014929238320231903, 1081076050061213697: 0

PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')

In [3]:
from serializer import Serializer

# Get starting users info

accounts = ["mizzaro"]
#for account in accounts:
 #   serializer = Serializer(f'check_data/{account}')
  #  profile = api.get_user(account)
   # profile_json = profile._json
    #serializer.serialize_json(f'{account}_profile.json', profile_json)

In [42]:
# Point #1 of the assignment: retrieve starting users followers and followings

for account in accounts:
    print(f"Processing @{account}")
    serializer = Serializer(f'check_data/{account}')

### FOLLOWERS

    account_followers = []
    for item in tweepy.Cursor(
            api.followers,
            screen_name=account,
            skip_status=True,
            include_user_entities=False
    ).items():
        found_follower = item._json
        account_followers.append(found_follower)

    print(f"Found {len(account_followers)} followers for @{account}")
    serializer.serialize_json(f"{account}_follower.json", account_followers)

### FOLLOWINGS

    account_followings = []
    for item in tweepy.Cursor(
            api.friends,
            screen_name=account,
            skip_status=True,
            include_user_entities=False
    ).items():
        found_followings = item._json
        account_followings.append(found_followings)

    print(f"@{account} follows {len(account_followings)} users")
    serializer.serialize_json(f"{account}_following.json", account_followings)

Processing @mizzaro
Found 158 followers for @mizzaro
Data serialized to path: check_data/mizzaro/mizzaro_follower.json


Rate limit reached. Sleeping for: 893


@mizzaro follows 331 users
Data serialized to path: check_data/mizzaro/mizzaro_following.json
Processing @KevinRoitero
Found 103 followers for @KevinRoitero
Data serialized to path: check_data/KevinRoitero/KevinRoitero_follower.json
@KevinRoitero follows 258 users
Data serialized to path: check_data/KevinRoitero/KevinRoitero_following.json


In [44]:
# Points #2 and #3 of the assignment: pick 5 random followers of the starting users,
# retrieve 10 followers each, pick 5 random followings of the starting users
# and retrieve 10 followings each

import random

for account in accounts:
    serializer = Serializer(f'check_data/{account}')
    json = serializer.read_json(f"{account}_follower.json")
    for count in range(0, 5):
        random_follower = random.choice(json)
        random_follower_screenName = random_follower["screen_name"]
        random_follower_id = random_follower["id"]
        random_follower_followers = []
        for item in tweepy.Cursor(
                api.followers,
                screen_name=random_follower_screenName,
                skip_status=True,
                include_user_entities=False
        ).items(10):
            found_follower = item._json
            random_follower_followers.append(found_follower)
        print(f"Found {len(random_follower_followers)} followers for @{random_follower_screenName}")
        serializer.serialize_json(f"random_{random_follower_id}_follower.json", random_follower_followers)

    json = serializer.read_json(f"{account}_following.json")
    for count in range(0, 5):
        random_following = random.choice(json)
        random_following_screenName = random_following["screen_name"]
        random_following_id = random_following["id"]
        random_following_followings = []
        for item in tweepy.Cursor(
                api.friends,
                screen_name=random_following_screenName,
                skip_status=True,
                include_user_entities=False
        ).items(10):
            found_friend = item._json
            random_following_followings.append(found_friend)
        print(f"@{random_following_screenName} follows {len(random_following_followings)} users")
        serializer.serialize_json(f"random_{random_following_id}_following.json", random_following_followings)

Data read from path: check_data/mizzaro/mizzaro_follower.json
Found 10 followers for @ntcirstc
Data serialized to path: check_data/mizzaro/random_2988561009_follower.json
Found 10 followers for @fmnardini
Data serialized to path: check_data/mizzaro/random_125483940_follower.json
Found 10 followers for @MauroLorenzutti
Data serialized to path: check_data/mizzaro/random_332134740_follower.json
Found 10 followers for @NewsIR_Workshop
Data serialized to path: check_data/mizzaro/random_4149032843_follower.json
Found 10 followers for @ICMR2014
Data serialized to path: check_data/mizzaro/random_1322931373_follower.json
Data read from path: check_data/mizzaro/mizzaro_following.json
@SabinaGuzzanti follows 10 users
Data serialized to path: check_data/mizzaro/random_16867526_following.json
@sarvk follows 10 users
Data serialized to path: check_data/mizzaro/random_384127183_following.json
@AcademiaObscura follows 10 users
Data serialized to path: check_data/mizzaro/random_2645602164_following.jso

In [45]:
# Point #4 of the assignment: retrieve all encountered users' profile
from datetime import datetime
import os

error_count = 0         # Keep trace of how many errors occurred during user retrieval (account not found)
duplicate_count = 0     # Keep trace of users already encountered

all_users = []
processed_ids = []

print(f"Start at {datetime.now()}")
for account in accounts:
    print(
        f'\n\n*************************************\nProcessing {account} and his friends\n*************************************')
    serializer = Serializer(f'check_data/{account}')
    with os.scandir(f'check_data/{account}') as it:
        for entry in it:
            if entry.name.endswith('.json') and not entry.name.endswith('profile.json'):
                print('\n\n******************')
                users_data = serializer.read_json(f"{entry.name}")
                print(f'\nProcessing {entry.name}, containing {len(users_data)} users\n******************\n\n')
                for user in users_data:
                    if user["id"] not in processed_ids:
                        try:
                            print(f'Processing {user["id"]}, user #{len(all_users) + 1}')
                            user_details = api.get_user(user["id"])._json
                            useful_user_details = {
                                "id": user_details["id"],
                                "name": user_details["name"],
                                "screen_name": user_details["screen_name"],
                                "description": user_details["description"],
                                "followers_count": user_details["followers_count"],
                                "friends_count": user_details["friends_count"],
                                "profile_image_url_https": user_details["profile_image_url_https"]
                            }
                            all_users.append(useful_user_details)
                            processed_ids.append(user_details["id"])
                        except tweepy.TweepError:
                            error_count += 1
                            print("Skipped user because of error")
                    else:
                        duplicate_count += 1
serializer = Serializer('check_data')
print('\n\n*************************************\n')
serializer.serialize_json(f"mizzaro_KevinRoitero_all_users.json", all_users)
print('\n*************************************\n\n')
print(f'Found {error_count} errors and {duplicate_count} duplicates')

Start at 2020-11-16 21:07:07.526618


*************************************
Processing mizzaro and his friends
*************************************


******************
Data read from path: check_data/mizzaro/mizzaro_follower.json

Processing mizzaro_follower.json, containing 158 users
******************


Processing 740410382, user #1
Processing 1972411447, user #2
Processing 257691637, user #3
Processing 4893681, user #4
Processing 116897811, user #5
Processing 551951577, user #6
Processing 91697719, user #7
Processing 94732055, user #8
Processing 514667085, user #9
Processing 4149032843, user #10
Processing 4188048952, user #11
Processing 14451127, user #12
Processing 2988561009, user #13
Processing 920700510306529280, user #14
Processing 950377242483343360, user #15
Processing 1527607790, user #16
Processing 923649456981135367, user #17
Processing 890413838578966528, user #18
Processing 1477918760, user #19
Processing 810893744593584128, user #20
Processing 125483940, user #21
Pro

Processing 569790116, user #242
Processing 337600744, user #243
Processing 34227622, user #244
Processing 2816046521, user #245
Processing 1891806212, user #246
Processing 33474334, user #247
Processing 1530894998, user #248
Processing 55858928, user #249
Processing 15247794, user #250
Processing 273561140, user #251
Processing 17707080, user #252
Processing 172260408, user #253
Processing 1237393394, user #254
Processing 48839631, user #255
Processing 2607616422, user #256
Processing 82774113, user #257
Processing 2422189058, user #258
Processing 2550133394, user #259
Processing 850107536, user #260
Processing 354847710, user #261
Processing 70266297, user #262
Processing 16689804, user #263
Processing 326912209, user #264
Processing 259403751, user #265
Processing 19540483, user #266
Processing 567360618, user #267
Processing 352053266, user #268
Processing 393956778, user #269
Processing 40156543, user #270
Processing 2438493248, user #271
Processing 2880905121, user #272
Processing

Processing 16507835, user #446
Processing 9271782, user #447
Processing 193168799, user #448


******************
Data read from path: check_data/mizzaro/random_384127183_following.json

Processing random_384127183_following.json, containing 10 users
******************


Processing 758459412, user #449
Processing 961367101553430530, user #450
Processing 505947709, user #451
Processing 1677625856, user #452
Processing 47318173, user #453
Processing 500728774, user #454
Processing 461202117, user #455
Processing 989371090701963265, user #456
Processing 301178769, user #457
Processing 1120554912574631937, user #458


******************
Data read from path: check_data/mizzaro/random_4149032843_follower.json

Processing random_4149032843_follower.json, containing 10 users
******************


Processing 808051518242979841, user #459
Processing 1174926313234124801, user #460
Processing 1190572258265161728, user #461
Processing 92344379, user #462
Processing 573460891, user #463
Processing 37

Processing 1344951, user #664
Processing 119711344, user #665
Processing 281448138, user #666
Processing 755953153, user #667
Processing 117662694, user #668
Processing 2097571, user #669
Processing 11348282, user #670
Processing 200163448, user #671
Processing 742143, user #672
Processing 2979365176, user #673
Processing 14293310, user #674
Processing 133790890, user #675
Processing 180463340, user #676
Processing 62513246, user #677
Processing 337886919, user #678
Processing 166739404, user #679
Processing 15254807, user #680
Processing 813286, user #681
Processing 5893702, user #682


******************
Data read from path: check_data/KevinRoitero/random_125483940_follower.json

Processing random_125483940_follower.json, containing 10 users
******************




******************
Data read from path: check_data/KevinRoitero/random_20356196_follower.json

Processing random_20356196_follower.json, containing 10 users
******************


Processing 1260993142708297730, user #683
Pro

In [ ]:
# As requested, before building the social network is necessary to check friendships

def get_friendship(sourceid, targetid, api):
    kind = ""

    friendship = api.show_friendship(source_id=sourceid, target_id=targetid)

    if not friendship[0].following and not friendship[0].followed_by:
        kind = "none"
    elif not friendship[0].following and friendship[0].followed_by:
        kind = "r_l"
    elif friendship[0].following and not friendship[0].followed_by:
        kind = "l_r"
    else:
        kind = "bi"

    return {
        "source_id": sourceid,
        "target_id": targetid,
        "friendship": kind
    }

serializer = Serializer('check_data')
users = serializer.read_json("unique_all_users.json")
edges = []
count = 0
for account in accounts:
    serializer = Serializer(f'check_data/{account}')
    account_json = serializer.read_json(f"{account}_profile.json")
    account_id = account_json["id"]
    for user in users:
        if user["id"] is not account_id:
            edges.append(get_friendship(account_id, user["id"], api))
            count += 1
            print(f"Added friendship between {account} and {user['screen_name']} #{count}")
    # There's also the necessity to check friendships between the random picked users at Point #2 and #3 and their followers and followings
    with os.scandir(f'check_data/{account}') as it:
        for entry in it:
            if entry.name.endswith('.json') and entry.name.startswith('random'):
                fileId = int(entry.name.split("_")[1])
                json = serializer.read_json(entry.name)
                for profile in json:
                    edges.append(get_friendship(fileId, profile["id"], api))
                    count += 1
                    print(f"Added friendship between {fileId} and {profile['screen_name']} #{count}")

serializer = Serializer('check_data')
serializer.serialize_json(f'mizzaro_all_friendships.json', edges)

Data read from path: check_data/unique_all_users.json
Data read from path: check_data/mizzaro/mizzaro_profile.json
Added friendship between mizzaro and Ilele99 #1
Added friendship between mizzaro and Duesynapse #2
Added friendship between mizzaro and ChriShot90 #3
Added friendship between mizzaro and MelissaVasi #4
Added friendship between mizzaro and NSkinis #5
Added friendship between mizzaro and JoeChevalierTw #6
Added friendship between mizzaro and rmit_csit #7
Added friendship between mizzaro and BiancolinoElisa #8
Added friendship between mizzaro and ZuzanPink #9
Added friendship between mizzaro and damiano10 #10
Added friendship between mizzaro and ResearchMig #11
Added friendship between mizzaro and Ester15361406 #12
Added friendship between mizzaro and LaSabbiaCheCant #13
Added friendship between mizzaro and GiorgioTassara #14
Added friendship between mizzaro and SolomanSangem #15
Added friendship between mizzaro and Tommaso_Dri #16
Added friendship between mizzaro and KevinRo

Added friendship between mizzaro and Carla_dv_ #152
Added friendship between mizzaro and MarrrconSara #153
Added friendship between mizzaro and VerenaAmbrosino #154
Added friendship between mizzaro and pavandrea #155
Added friendship between mizzaro and Romagna21 #156
Added friendship between mizzaro and matteo_comello #157
Added friendship between mizzaro and RobyBello91 #158
Added friendship between mizzaro and filippo_missio #159
Added friendship between mizzaro and LeonardoSialino #160
Added friendship between mizzaro and heartlinemusic #161
Added friendship between mizzaro and Pischiaz #162
Added friendship between mizzaro and WojtekStrzelec #163
Added friendship between mizzaro and TassaraCata #164
Added friendship between mizzaro and LaPetkova #165
Added friendship between mizzaro and OperaStarz #166
Added friendship between mizzaro and GTT_it #167
Added friendship between mizzaro and MDCC91 #168
Added friendship between mizzaro and LazarusCRY #169
Added friendship between mizza

Rate limit reached. Sleeping for: 825


Added friendship between mizzaro and GilGalad88 #181
Added friendship between mizzaro and GiusySivilotti #182
Added friendship between mizzaro and CiaoMmmiky #183
Added friendship between mizzaro and D_Oblivion12 #184
Added friendship between mizzaro and CreativeElle #185
Added friendship between mizzaro and tanya_931 #186
Added friendship between mizzaro and ThomPollo93 #187
Added friendship between mizzaro and vendettapirati #188
Added friendship between mizzaro and cristinalisotto #189
Added friendship between mizzaro and tiritwittoio #190
Added friendship between mizzaro and LindaDalMas #191
Added friendship between mizzaro and BoopLucy #192
Added friendship between mizzaro and SoulSister95 #193
Added friendship between mizzaro and CARACASPARTY1 #194
Added friendship between mizzaro and postinoindie #195
Added friendship between mizzaro and SimiBunny #196
Added friendship between mizzaro and GLINTonline #197
Added friendship between mizzaro and uniud #198
Added friendship between m

Added friendship between mizzaro and eglu81 #333
Added friendship between mizzaro and cikm2020 #334
Added friendship between mizzaro and zerocalcare #335
Added friendship between mizzaro and Halo #336
Added friendship between mizzaro and DAVIDPARENZO #337
Added friendship between mizzaro and Bungie #338
Added friendship between mizzaro and DelReyStarWars #339
Added friendship between mizzaro and PosteSpedizioni #340
Added friendship between mizzaro and IAmOxfordComma #341
Added friendship between mizzaro and HIDEO_KOJIMA_EN #342
Added friendship between mizzaro and PlayApex #343
Added friendship between mizzaro and startrekcbs #344
Added friendship between mizzaro and talkingkotlin #345
Added friendship between mizzaro and AbN0ri #346
Added friendship between mizzaro and PaoloGentiloni #347
Added friendship between mizzaro and gerardopandolfi #348
Added friendship between mizzaro and julian_gollop #349
Added friendship between mizzaro and Phoenix_Point #350
Added friendship between miz

...

Added friendship between mizzaro and OmarMonestier #3754
Added friendship between mizzaro and Yahoo #3755
Added friendship between mizzaro and concitadeg #3756
Added friendship between mizzaro and vardi #3757
Added friendship between mizzaro and RuSSIR #3758
Added friendship between mizzaro and CaRRws #3759
Added friendship between mizzaro and fortnow #3760
Added friendship between mizzaro and duncanjwatts #3761
Added friendship between mizzaro and roiblanco #3762
Added friendship between mizzaro and lemire #3763
Added friendship between mizzaro and asmeaton #3764
Added friendship between mizzaro and seawan #3765
Added friendship between mizzaro and richardm_ #3766
Added friendship between mizzaro and bpiwowar #3767
Added friendship between mizzaro and abdur #3768
Added friendship between mizzaro and rodrygo_santos #3769
Added friendship between mizzaro and IIiX2014 #3770
Added friendship between mizzaro and krisztianbalog #3771
Added friendship between mizzaro and craig_macdonald #377

Rate limit reached. Sleeping for: 849


Added friendship between mizzaro and joemonj #3781
Added friendship between mizzaro and delsweil #3782
Added friendship between mizzaro and ictir2013 #3783
Added friendship between mizzaro and uniud #3784
Added friendship between mizzaro and ManuelaCroatto #3785
Added friendship between mizzaro and ldigaspero #3786
Added friendship between mizzaro and peterpaws #3787
Added friendship between mizzaro and wesselkraaij #3788
Added friendship between mizzaro and tonygrr #3789
Added friendship between mizzaro and AppenGlobal #3790
Added friendship between mizzaro and claclarke #3791
Added friendship between mizzaro and donadeb #3792
Added friendship between mizzaro and UNED #3793
Added friendship between mizzaro and NTCIR #3794
Added friendship between mizzaro and bing #3795
Added friendship between mizzaro and marcotravaglio #3796
Added friendship between mizzaro and spinozait #3797
Added friendship between mizzaro and djoerd #3798
Added friendship between mizzaro and HCIR_GeneG #3799
Adde

Added friendship between mizzaro and jayzhang_syd #3932
Added friendship between mizzaro and pascalguyon #3933
Added friendship between mizzaro and JTrippas #3934
Added friendship between mizzaro and HLForum #3935
Added friendship between mizzaro and pchandarr #3936
Added friendship between mizzaro and HCjnl #3937
Added friendship between mizzaro and amrapaliz #3938
Added friendship between mizzaro and Perangur #3939
Added friendship between mizzaro and hiijjournal1 #3940
Added friendship between mizzaro and UjLaw #3941
Added friendship between mizzaro and DGarigliotti #3942
Added friendship between mizzaro and ZorroLovesLab #3943
Added friendship between mizzaro and SpringerIRJ #3944
Added friendship between mizzaro and alex_checco #3945
Added friendship between mizzaro and BrazierPhD #3946
Added friendship between mizzaro and ESSAYWRITERD #3947
Added friendship between mizzaro and medardAttoukou #3948
Added friendship between mizzaro and ZaBi94 #3949
Added friendship between mizzaro 

Rate limit reached. Sleeping for: 842


In [7]:
import serializer
import pandas as pd
serializer = serializer.Serializer("check_data")
json = serializer.read_json("mizzaro_all_friendships_OLD.json")
json_unique = pd.DataFrame(json).drop_duplicates().to_dict('records')
serializer.serialize_json("mizzaro_all_friendships.json", json_unique)
print(len(json))
print(len(json_unique))

Data read from path: check_data/mizzaro_all_friendships_OLD.json
Data serialized to path: check_data/mizzaro_all_friendships.json
4320
3317
